*Chunks in Pandas, PyArrow, and Parquet - Amex Competition*

![](dalle.jpg)

In [56]:
# libraries
import pandas as pd
import numpy as np 
import psutil
import gc

import pyarrow.parquet as pq
import pyarrow as pa

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
pd.options.display.max_info_columns = 300

In [25]:
def available_memory_gb():
    return psutil.virtual_memory().available / (1024**3)

## Introduction - The AMEX Default Prediction Competition

The AMEX default prediction competition in Kaggle is a binary classification problem where the goal is to predict whether a credit card holder will default on their payment within 120 days after the latest credit card statement based on data from a 18-month performance window after the latest credit card statement. 

The first step in any data science project is to load the data and understand its structure. The problem here is that the dataset is 50GB in size and it is not possible to load it all into memory at once (at least not in my machine). So here is where headaches start to appear.

In this notebook, I will show how to load the data in chunks using Pandas and PyArrow, and how to save it in Parquet format. This way, we can load the data in chunks and avoid memory issues.

## You don't need to load the data all at once

The first thing to understand is that you don't need to load the data all at once. You can have a first glance to the data by loading only a small sample of it. This will give you an idea of the data structure and help you to decide what to do next. 



In [31]:
PATH = '/home/jmanu/git/amex_competition/train_data.csv'
data = pd.read_csv(PATH, nrows=35000)
data.head(5)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,NaN,NaN,0.000630,0.080986,0.708906,0.170600,0.006204,0.358587,0.525351,0.255736,NaN,0.063902,0.059416,0.006466,0.148698,1.335856,0.008207,0.001423,0.207334,0.736463,0.096219,NaN,0.023381,0.002768,0.008322,1.001519,0.008298,0.161345,0.148266,0.922998,0.354596,0.152025,0.118075,0.001882,0.158612,0.065728,0.018385,0.063646,0.199617,0.308233,0.016361,0.401619,0.091071,CR,O,0.007126,0.007665,NaN,0.652984,0.008520,NaN,0.004730,6.0,0.272008,0.008363,0.515222,0.002644,0.009013,0.004808,0.008342,0.119403,0.004802,0.108271,0.050882,NaN,0.007554,0.080422,0.069067,NaN,0.004327,0.007562,NaN,0.007729,0.000272,0.001576,0.004239,0.001434,NaN,0.002271,0.004061,0.007121,0.002456,0.002310,0.003532,0.506612,0.008033,1.009825,0.084683,0.003820,0.007043,0.000438,0.006452,0.000830,0.005055,NaN,0.0,0.005720,0.007084,NaN,0.000198,0.008907,NaN,1,0.002537,0.005177,0.006626,0.009705,0.007782,0.002450,1.001101,0.002665,0.007479,0.006893,1.503673,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.894090,0.135561,0.911191,0.974539,0.001243,0.766688,1.008691,1.004587,0.893734,NaN,0.670041,0.009968,0.004572,NaN,1.008949,2.0,NaN,0.004326,NaN,NaN,NaN,1.007336,0.210060,0.676922,0.007871,1.0,0.238250,0.0,4.0,0.232120,0.236266,0.0,0.702280,0.434345,0.003057,0.686516,0.008740,1.0,1.003319,1.007819,1.000080,0.006805,NaN,0.002052,0.005972,NaN,0.004345,0.001535,NaN,NaN,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,NaN,NaN,0.002526,0.069419,0.712795,0.113239,0.006206,0.353630,0.521311,0.223329,NaN,0.065261,0.057744,0.001614,0.149723,1.339794,0.008373,0.001984,0.202778,0.720886,0.099804,NaN,0.030599,0.002749,0.002482,1.009033,0.005136,0.140951,0.143530,0.919414,0.326757,0.156201,0.118737,0.001610,0.148459,0.093935,0.013035,0.065501,0.151387,0.265026,0.017688,0.406326,0.086805,CR,O,0.002413,0.007148,NaN,0.647093,0.002238,NaN,0.003879,6.0,0.188970,0.004030,0.509048,0.004193,0.007842,0.001283,0.006524,0.140611,0.000094,0.101018,0.040469,NaN,0.004832,0.081413,0.074166,NaN,0.004203,0.005304,NaN,0.001864,0.000979,0.009896,0.007597,0.000509,NaN,0.009810,0.000127,0.005966,0.000395,0.001327,0.007773,0.500855,0.000760,1.009461,0.081843,0.000347,0.007789,0.004311,0.002332,0.009469,0.003753,NaN,0.0,0.007584,0.006677,NaN,0.001142,0.005907,NaN,1,0.008427,0.008979,0.001854,0.009924,0.005987,0.002247,1.006779,0.002508,0.006827,0.002837,1.503577,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.902135,0.136333,0.919876,0.975624,0.004561,0.786007,1.000084,1.004118,0.906841,NaN,0.668647,0.003921,0.004654,NaN,1.003205,2.0,NaN,0.008707,NaN,NaN,NaN,1.007653,0.184093,0.822281,0.003444,1.0,0.247217,0.0,4.0,0.243532,0.241885,0.0,0.707017,0.430501,0.001306,0.686414,0.000755,1.0,1.008394,1.004333,1.008344,0.004407,NaN,0.001034,0.004838,NaN,0.

Using the `info()` method, you can have a better sense of how the dataset is structured.

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 190 columns):
 #    Column       Non-Null Count  Dtype  
---   ------       --------------  -----  
 0    customer_ID  35000 non-null  object 
 1    S_2          35000 non-null  object 
 2    P_2          34690 non-null  float64
 3    D_39         35000 non-null  float64
 4    B_1          35000 non-null  float64
 5    B_2          34994 non-null  float64
 6    R_1          35000 non-null  float64
 7    S_3          28053 non-null  float64
 8    D_41         34994 non-null  float64
 9    B_3          34994 non-null  float64
 10   D_42         4889 non-null   float64
 11   D_43         24217 non-null  float64
 12   D_44         33279 non-null  float64
 13   B_4          35000 non-null  float64
 14   D_45         34994 non-null  float64
 15   B_5          35000 non-null  float64
 16   R_2          35000 non-null  float64
 17   D_46         27411 non-null  float64
 18   D_47         35000 non-n

In [33]:
print(f'Number of data types object cols: {data.select_dtypes("object").columns.size}')
print(f'Number of data types int cols: {data.select_dtypes("int64").columns.size}')
print(f'Number of data types float cols: {data.select_dtypes("float64").columns.size}')

Number of data types object cols: 4
Number of data types int cols: 1
Number of data types float cols: 185


## Data preprocessing
### `customer_ID` hurts your eyes!
When you execute the head method in the dataframe, the `customer_ID` column hurts your eyes. It's a column that store strings that appears to be a hexadecimal. This type of string typically represents binary data in a human-readable format, using the characters 0-9 and a-f (or A-F) to represent the values 0 through 15 in each digit. Given its length, it is likely that this column is a hash of some sort. Given its length (64 hexadecimal characters), this string is likely a SHA-256 hash. 

This mean that each row uses 64 bytes of memory. So following Chris Deotte's advice [here](https://www.kaggle.com/competitions/amex-default-prediction/discussion/328054) and [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/308635), we are going to take the last 16 digits of the hexadecimal string and convert it to an integer.

In [34]:
data['customer_ID'] = data['customer_ID'].apply(lambda x: int(x[-16:], 16)).astype('int32')

### From 11 categorical columns to 13 categorical columns
American Express has also provide a list of categorical features that we can convert to categorical type in Pandas. This will reduce the memory usage of these columns. The list is below:
```	
['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
```

In [35]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
data[cat_cols].head(5)

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68
0,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
1,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
2,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
3,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
4,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0


In [36]:
data[cat_cols].nunique()

B_30     3
B_38     7
D_114    2
D_116    2
D_117    7
D_120    2
D_126    3
D_63     6
D_64     4
D_66     2
D_68     7
dtype: int64

To convert a columns to categorical type, you can use the following code:

In [37]:
data[cat_cols] = data[cat_cols].astype('category')

In [38]:
for cat in cat_cols:
    data[cat] = data[cat].cat.codes

Among the numerical columns, we also discover that there are some columns that are categorical. We can convert them to categorical type in Pandas. This will reduce the memory usage of these columns

In [39]:
for col in data.select_dtypes(include='number'):
    if data[col].nunique() < 3:
        print('Columns with less than 2 unique values', data[col].name, data[col].dtype)

Columns with less than 2 unique values D_87 float64
Columns with less than 2 unique values B_31 int64


In [40]:
data[['D_87', 'B_31']].value_counts()

D_87  B_31
1.0   1       29
      0        6
Name: count, dtype: int64

In [41]:
for col in ['D_87', 'B_31']:
    data[col] = data[col].fillna(-1)
    data[col] = data[col].astype('int8')

### Convert `S_2` to a `datetime` data type

The `S_2` column is a date column that is stored as a string. We can convert it to a datetime data type in Pandas using the following code:

In [42]:
data['S_2'] = pd.to_datetime(data['S_2'], format='%Y-%m-%d')

This will reduce the memory usage of this column but we can go further and split the date into year, month, and day columns. This will allow us to analyze the data by year, month, and day, and will also reduce the memory usage even further. 

In [43]:
# Split into separate columns
data['S_2_year'] = (data['S_2'].dt.year % 100).astype('int8')
data['S_2_month'] = data['S_2'].dt.month.astype('int8')
data['S_2_day'] = data['S_2'].dt.day.astype('int8')
del data['S_2']

### 185 columns `float64` to `float32` or `float16`

The first thing you notice when you inspect the `info()` method is that we have 185 float64 columns which typically consume 64 bits, or 8 bytes per element. This type of data type is used to represent double precision floating point numbers, which are numbers that have very large or very small magnitude and/or require a significant number of digits of precision. As we are going to learn latter, we can reduce the memory usage by changing the data type of these columns to `float32` and even `float16`.

In [44]:
len(data.select_dtypes(include=np.float64).columns)

175


After all the transformation so far, we reduced the number to 175 columns. If we change the data type of these columns to `float32`, we will reduce the memory usage by half. If we change the data type to `float16`, we will reduce the memory usage by 75%. 

So converting from `float64` to `float32` you can save 50% of memory usage without a significant loss in precision for many applications. 

As Chris Deotte pointed out in the notebook previously mentioned, They discovered that we can convert these to `float16` since the data has added uniform noise. This means that the data is not as precise as it seems. This can be also done for privacy protection. 

In [45]:
for col in data.select_dtypes(include=np.float64).columns:
    data[col] = data[col].fillna(-1).astype('float16')

## Chunk strategy

Once we've thoroughly understood our dataset and identified all necessary transformations to make it more manageable, the next step is to apply these transformations to the entire dataset. To avoid memory issues, we will process the data in chunks, similar to how batches are used in deep learning. Therefore, it's essential to develop a strategy for determining the sizes of these chunks. We will calculate the memory consumption per row based on our original sample data, allowing us to optimize the chunk size effectively.


Memory consumption for 35,000 rows: 59.81 MB

In [47]:
data = pd.read_csv(PATH, nrows=35000)
data.memory_usage(deep=True).sum() / 1024**2

59.81640434265137

Memory usage per row is: $\frac{59.81}{35000}=0.0017$ MB

In [49]:
(data.memory_usage(deep=True).sum() / 1024**2)/data.shape[0]

0.0017090401240757534

In order to have enough memory we are going to consider to use half of the memory available.

Target memory usage aprox: $\frac{8.26}{2}=4.15$ GB

In [50]:
available_memory_gb() / 2 

4.147722244262695

Chunk size in rows: $\frac{4.15}{0.0017}=2431508$ rows


In [51]:
(available_memory_gb() / 2) *1000 /  ((data.memory_usage(deep=True).sum() / 1024**2)/data.shape[0])

2431507.734935815

A more conservative approach would be to scale down this estimate by a factor to ensure smooth processing.

In [52]:
2431507.734935815 * 0.1

243150.7734935815

Our final chunk size will be 243151 rows.

In [53]:
del data
gc.collect()

1194

## Introduction to PyArrow and Parquet

PyArrow is a Python library that provides a bridge between Python and the Apache Arrow in-memory data format.

Apache Arrow is designed to improve the performance and efficiency of data by providing a standardized in-memory columnar data format that can be shared between different systems and languages. PyArrow includes support for reading and writing Parquet files.

`import pyarrow.parquet as pq` -> Imports the PyArrow library's Parquet module, which provides tools for reading and writing Parquet files

`import pyarrow as pa` ->  Imports the PyArrow library to access data structures like Tables, which are used to interface with Parquet files and other Arrow functionalities.

Once the libraries have been imported, we need to define the path where the Parquet file is located. Also we need to initialize the `writer`variable to `None` that will be used to create a ParquetWriter object, which will be used to write the data to the Parquet file.

In [54]:
output_path = 'train.parquet'
writer = None


Based on our estimation of memory usage, we set the chunksize to 243151. This is going to control how much data is read into memory at once. We can then iterate over the chunks of the DataFrame and write them to the Parquet file.

Next we can start to apply all the transformations to the data.

```python
chunksize = 243151
for chunk in pd.read_csv(PATH, chunksize=chunksize, engine='c'):
    # Transforming customer_ID
    chunk['customer_ID'] = chunk['customer_ID'].apply(lambda x: int(x[-16:], 16)).astype('int32')
    # Apply datetime transformations 
    chunk['S_2'] = pd.to_datetime(chunk['S_2'], format='%Y-%m-%d')
    chunk['year_last_2_digits'] = chunk['S_2'].dt.year % 100
    chunk['month'] = chunk['S_2'].dt.month
    chunk['day'] = chunk['S_2'].dt.day
    chunk[['year_last_2_digits', 'month', 'day']] = chunk[['year_last_2_digits', 'month', 'day']].astype('int8')
    # Apply category transformations
    chunk[cat_cols] = chunk[cat_cols].astype('category')
    for cat in cat_cols:
        chunk[cat] = chunk[cat].cat.codes.astype('int8')
    # Additional columns to convert to 'int8'
    for col in ['D_87', 'B_31']:
        chunk[col] = chunk[col].fillna(-1).astype('int8')
        
     # Convert float64 columns to float16 for all floating type columns
    for col in chunk.select_dtypes(include=np.float64).columns:
        chunk[col] = chunk[col].fillna(-1).astype('float16')
````

Inside of the loop, we need to convert the the `chunk` dataframe into a PyArrow Table object using the `pa.Table.from_pandas()` method. This method takes the DataFrame as input and returns a Table object that can be written to a Parquet file.

```python
# Convert DataFrame to PyArrow Table
table = pa.Table.from_pandas(chunk)
```

Now we need to initialize the ParquetWriter object if it is not already initialized. We can do this by checking if the `writer` variable is `None`. If it is, we create a new ParquetWriter object using the `pq.ParquetWriter()` method. This method takes the output path and the schema of the data as input.

```python
if writer is None:
        writer = pq.ParquetWriter(output_path, table.schema, compression='snappy')
```

Finally the next line of code, writes the PyArrow Table object to the Parquet file using the `writer.write_table()` method. This is done incrementally for each chunk of data read from the CSV file.

```python
writer.write_table(table)
```

After the loop is finished, we need to close the ParquetWriter object using the `writer.close()` method. This will finalize the writing process and save the data to the Parquet file.

```python
if writer:
    writer.close()
```

This is a simple example of how to convert a large CSV file to a Parquet file using PyArrow. This process can be customized further based on the specific requirements of the data and the desired output format.

## Running the whole code

Let's run the whole code to get our final train set in Parquet format.

In [57]:
# Define the path for the output Parquet file
output_path = 'train.parquet'
# Initialize PyArrow Parquet writer, initially without a schema
writer = None

# Read the CSV file in chunks
chunksize = 243151  # Customized to your available memory and dataset size

# Cat columns 
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

for chunk in pd.read_csv(PATH, chunksize=chunksize, na_values=-1, engine='c'):
    # Transforming customer_ID
    chunk['customer_ID'] = chunk['customer_ID'].apply(lambda x: int(x[-16:], 16)).astype('int32')
    # Apply datetime transformations 
    chunk['S_2'] = pd.to_datetime(chunk['S_2'], format='%Y-%m-%d')
    chunk['year_last_2_digits'] = chunk['S_2'].dt.year % 100
    chunk['month'] = chunk['S_2'].dt.month
    chunk['day'] = chunk['S_2'].dt.day
    chunk[['year_last_2_digits', 'month', 'day']] = chunk[['year_last_2_digits', 'month', 'day']].astype('int8')
    # Apply category transformations
    chunk[cat_cols] = chunk[cat_cols].astype('category')
    for cat in cat_cols:
        chunk[cat] = chunk[cat].cat.codes.astype('int8')
    # Additional columns to convert to 'int8'
    for col in ['D_87', 'B_31']:
        chunk[col] = chunk[col].fillna(-1).astype('int8')
        
     # Convert float64 columns to float16 for all floating type columns
    for col in chunk.select_dtypes(include=np.float64).columns:
        chunk[col] = chunk[col].fillna(-1).astype('float16')
    
    # Convert DataFrame to PyArrow Table
    table = pa.Table.from_pandas(chunk)
    
    # Initialize writer with schema from the first chunk
    if writer is None:
        writer = pq.ParquetWriter(output_path, table.schema, compression='snappy')
    
    # Write table to Parquet file
    writer.write_table(table)

# Don't forget to close the writer to finalize the Parquet file
if writer:
    writer.close()
